# Hotel Booking Analysis: Advanced Analytics and Business Impact Study

## Project Overview
This analysis explores hotel booking patterns and cancellation behaviors to optimize revenue and operational efficiency. Using advanced analytics and machine learning techniques, we aim to provide actionable insights for strategic decision-making.

## Business Objectives
1. Identify key factors driving booking cancellations
2. Develop customer segmentation profiles
3. Quantify revenue impact of cancellations
4. Create predictive models for early cancellation detection
5. Propose data-driven strategies for revenue optimization

## Methodology
1. Data Quality Assessment and Preprocessing
2. Advanced Feature Engineering
3. Statistical Analysis and Hypothesis Testing
4. Customer Segmentation Analysis
5. Predictive Modeling
6. Business Impact Evaluation

## Expected Deliverables
- Comprehensive cancellation risk profiles
- Revenue impact analysis by segment
- ML model for cancellation prediction
- Actionable recommendations for management

In [34]:
# 1. Environment Setup and Configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
import logging
import os

# Machine Learning Libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import xgboost as xgb
import lightgbm as lgb

# Visualization Settings
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['axes.grid'] = True

# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('hotel_analysis.log')
    ]
)
logger = logging.getLogger(__name__)

# Suppress Warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Create Output Directories
output_dirs = ['outputs/visuals', 'outputs/models', 'outputs/reports']
for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)

logger.info("Environment setup completed successfully")

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# 2. Data Loading and Quality Assessment
def load_and_validate_data(file_path):
    """
    Load and perform initial validation of the dataset.
    
    Parameters:
    -----------
    file_path : str
        Path to the hotel bookings dataset
        
    Returns:
    --------
    pd.DataFrame
        Validated and initially processed dataset
    """
    try:
        # Load the dataset
        logger.info(f"Loading dataset from {file_path}")
        df = pd.read_csv(file_path)
        
        # Initial data quality checks
        logger.info("\nPerforming initial data quality checks:")
        logger.info(f"Shape of dataset: {df.shape}")
        logger.info(f"\nMissing values:\n{df.isnull().sum()}")
        logger.info(f"Duplicate rows: {df.duplicated().sum()}")
        
        # Data type validation
        logger.info("\nData types of columns:")
        logger.info(df.dtypes)
        
        # Basic statistics
        logger.info("\nBasic statistics of numerical columns:")
        logger.info(df.describe())
        
        return df
    
    except Exception as e:
        logger.error(f"Error loading dataset: {str(e)}")
        raise

# Load the dataset
data = load_and_validate_data('data/raw/hotel_bookings.csv')

# Display sample of the data
display(data.head())

## 3. Data Preprocessing and Cleaning

### Objectives:
1. Handle missing values appropriately
2. Remove duplicates and anomalies
3. Correct data types and formats
4. Handle outliers
5. Ensure data consistency

### Approach:
- Use domain knowledge for missing value imputation
- Implement robust outlier detection
- Document all cleaning decisions for reproducibility

In [ ]:
# 3.1 Data Cleaning Functions
def analyze_missing_values(df):
    """
    Analyze missing values and their patterns.
    """
    # Calculate missing value statistics
    missing_stats = pd.DataFrame({
        'Missing_Count': df.isnull().sum(),
        'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
    }).sort_values('Missing_Percentage', ascending=False)
    
    logger.info("\nMissing Value Analysis:")
    logger.info(missing_stats)
    
    return missing_stats

def detect_outliers(df, columns):
    """
    Detect outliers using IQR method.
    """
    outliers_dict = {}
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
        outliers_dict[column] = len(outliers)
    
    logger.info("\nOutlier Detection Results:")
    logger.info(outliers_dict)
    
    return outliers_dict

def handle_missing_values(df):
    """
    Handle missing values using appropriate strategies.
    """
    df_clean = df.copy()
    
    # Children: Fill with 0 (assumption: missing means no children)
    df_clean['children'] = df_clean['children'].fillna(0)
    
    # Country: Fill with 'Unknown'
    df_clean['country'] = df_clean['country'].fillna('Unknown')
    
    # Agent: Fill with 0 (direct booking)
    df_clean['agent'] = df_clean['agent'].fillna(0)
    
    # Company: Fill with 0 (no company association)
    df_clean['company'] = df_clean['company'].fillna(0)
    
    logger.info("\nMissing values handled successfully")
    return df_clean

def clean_data(df):
    """
    Main data cleaning function.
    """
    try:
        logger.info("Starting data cleaning process...")
        
        # 1. Analyze missing values
        missing_stats = analyze_missing_values(df)
        
        # 2. Handle missing values
        df_clean = handle_missing_values(df)
        
        # 3. Remove duplicates
        duplicates = df_clean.duplicated().sum()
        df_clean = df_clean.drop_duplicates()
        logger.info(f"\nRemoved {duplicates} duplicate rows")
        
        # 4. Detect outliers in numerical columns
        numerical_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
        outliers = detect_outliers(df_clean, numerical_cols)
        
        # 5. Convert data types
        df_clean['arrival_date'] = pd.to_datetime(
            df_clean['arrival_date_year'].astype(str) + '-' +
            df_clean['arrival_date_month'] + '-' +
            df_clean['arrival_date_day_of_month'].astype(str)
        )
        
        # 6. Drop unnecessary columns
        cols_to_drop = ['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']
        df_clean = df_clean.drop(columns=cols_to_drop)
        
        logger.info("Data cleaning completed successfully")
        return df_clean
        
    except Exception as e:
        logger.error(f"Error in data cleaning: {str(e)}")
        raise

# Apply data cleaning
data_clean = clean_data(data)

# Display cleaning results
print("\nDataset shape after cleaning:", data_clean.shape)
print("\nData types after cleaning:")
print(data_clean.dtypes)
print("\nSample of cleaned data:")
display(data_clean.head())

In [ ]:
# 3.2 Data Quality Validation
def validate_data_quality(df):
    """
    Validate the quality of cleaned data.
    """
    validation_results = {
        'missing_values': df.isnull().sum().sum(),
        'duplicates': df.duplicated().sum(),
        'negative_values': {
            col: (df[col] < 0).sum() 
            for col in df.select_dtypes(include=['float64', 'int64']).columns
        }
    }
    
    # Check date ranges
    validation_results['date_range'] = {
        'min_date': df['arrival_date'].min(),
        'max_date': df['arrival_date'].max()
    }
    
    # Validate categorical variables
    categorical_cols = df.select_dtypes(include=['object']).columns
    validation_results['categorical_counts'] = {
        col: df[col].nunique() 
        for col in categorical_cols
    }
    
    return validation_results

# Perform validation
validation_results = validate_data_quality(data_clean)

# Display validation results
print("\nData Quality Validation Results:")
print("--------------------------------")
print(f"Missing Values: {validation_results['missing_values']}")
print(f"Duplicates: {validation_results['duplicates']}")
print("\nNegative Values Check:")
for col, count in validation_results['negative_values'].items():
    if count > 0:
        print(f"- {col}: {count} negative values")
print("\nDate Range:")
print(f"- From: {validation_results['date_range']['min_date']}")
print(f"- To: {validation_results['date_range']['max_date']}")
print("\nCategorical Variable Counts:")
for col, count in validation_results['categorical_counts'].items():
    print(f"- {col}: {count} unique values")

## 4. Feature Engineering and Transformation

### Objectives:
1. Create meaningful temporal features
2. Generate price-related metrics
3. Develop guest composition features
4. Engineer market segment indicators
5. Create interaction features

### Approach:
- Focus on business-relevant feature creation
- Ensure features are interpretable
- Document feature importance and rationale

In [ ]:
# 4.1 Feature Engineering Functions
class FeatureEngineer:
    """
    A class to handle all feature engineering operations.
    """
    def __init__(self, df):
        self.df = df.copy()
        self.logger = logging.getLogger(__name__)
    
    def create_temporal_features(self):
        """Create time-based features"""
        self.df['booking_year'] = self.df['arrival_date'].dt.year
        self.df['booking_month'] = self.df['arrival_date'].dt.month
        self.df['booking_day'] = self.df['arrival_date'].dt.day
        self.df['booking_dayofweek'] = self.df['arrival_date'].dt.dayofweek
        self.df['is_weekend_arrival'] = self.df['booking_dayofweek'].isin([5, 6]).astype(int)
        self.df['is_peak_season'] = self.df['booking_month'].isin([7, 8, 12]).astype(int)
        
        return self
    
    def create_price_features(self):
        """Create price-related features"""
        self.df['total_nights'] = self.df['stays_in_weekend_nights'] + self.df['stays_in_week_nights']
        self.df['total_cost'] = self.df['adr'] * self.df['total_nights']
        self.df['avg_price_per_person'] = self.df['total_cost'] / (self.df['adults'] + self.df['children'] + self.df['babies'])
        self.df['is_high_price'] = (self.df['adr'] > self.df['adr'].mean()).astype(int)
        
        return self
    
    def create_guest_features(self):
        """Create guest composition features"""
        self.df['total_guests'] = self.df['adults'] + self.df['children'] + self.df['babies']
        self.df['has_children'] = ((self.df['children'] > 0) | (self.df['babies'] > 0)).astype(int)
        self.df['is_family'] = ((self.df['adults'] >= 2) & (self.df['has_children'] == 1)).astype(int)
        self.df['is_single'] = ((self.df['adults'] == 1) & (self.df['has_children'] == 0)).astype(int)
        
        return self
    
    def create_booking_features(self):
        """Create booking-related features"""
        self.df['is_long_stay'] = (self.df['total_nights'] > 7).astype(int)
        self.df['is_long_lead_time'] = (self.df['lead_time'] > 90).astype(int)
        self.df['has_special_requests'] = (self.df['total_of_special_requests'] > 0).astype(int)
        self.df['is_repeated_guest'] = self.df['is_repeated_guest'].astype(int)
        
        return self
    
    def create_market_features(self):
        """Create market segment features"""
        self.df['is_direct_booking'] = (self.df['market_segment'] == 'Direct').astype(int)
        self.df['is_corporate'] = (self.df['market_segment'] == 'Corporate').astype(int)
        
        # One-hot encode market segment and meal
        market_dummies = pd.get_dummies(self.df['market_segment'], prefix='market')
        meal_dummies = pd.get_dummies(self.df['meal'], prefix='meal')
        self.df = pd.concat([self.df, market_dummies, meal_dummies], axis=1)
        
        return self
    
    def transform_all_features(self):
        """Apply all feature transformations"""
        self.logger.info("Starting feature engineering process...")
        
        (self.create_temporal_features()
             .create_price_features()
             .create_guest_features()
             .create_booking_features()
             .create_market_features())
        
        self.logger.info("Feature engineering completed successfully")
        return self.df

# Apply feature engineering
feature_engineer = FeatureEngineer(data_clean)
data_featured = feature_engineer.transform_all_features()

# Display new features summary
print("\nNew Features Created:")
print("--------------------")
new_features = set(data_featured.columns) - set(data_clean.columns)
print(f"Total new features created: {len(new_features)}")
print("\nSample of new features:")
print(sorted(list(new_features))[:10])

# Display sample of transformed data
print("\nSample of transformed data:")
display(data_featured.head())

## 5. Statistical Analysis

### Objectives:
1. Understand distributions and relationships
2. Test hypotheses about booking patterns
3. Identify significant correlations
4. Analyze seasonal patterns
5. Evaluate cancellation factors

### Approach:
- Rigorous statistical testing
- Clear visualization of results
- Focus on business implications

In [ ]:
# 5.1 Statistical Analysis Functions
class StatisticalAnalyzer:
    def __init__(self, df):
        self.df = df
        self.logger = logging.getLogger(__name__)
    
    def run_hypothesis_tests(self):
        """Perform statistical hypothesis tests"""
        results = {}
        
        # Test 1: Is there a significant difference in ADR between canceled and non-canceled bookings?
        t_stat, p_value = stats.ttest_ind(
            self.df[self.df['is_canceled'] == 1]['adr'],
            self.df[self.df['is_canceled'] == 0]['adr']
        )
        results['adr_cancellation'] = {'t_statistic': t_stat, 'p_value': p_value}
        
        # Test 2: Chi-square test for independence between market segment and cancellation
        contingency = pd.crosstab(self.df['market_segment'], self.df['is_canceled'])
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
        results['market_cancellation'] = {'chi2': chi2, 'p_value': p_value}
        
        return results
    
    def analyze_correlations(self):
        """Analyze correlations between numerical variables"""
        numerical_cols = self.df.select_dtypes(include=['float64', 'int64']).columns
        correlation_matrix = self.df[numerical_cols].corr()
        
        # Find strong correlations
        strong_correlations = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i):
                if abs(correlation_matrix.iloc[i, j]) > 0.5:
                    strong_correlations.append({
                        'var1': correlation_matrix.columns[i],
                        'var2': correlation_matrix.columns[j],
                        'correlation': correlation_matrix.iloc[i, j]
                    })
        
        return correlation_matrix, strong_correlations
    
    def analyze_seasonality(self):
        """Analyze seasonal patterns"""
        seasonal_stats = self.df.groupby('booking_month').agg({
            'is_canceled': 'mean',
            'adr': 'mean',
            'total_guests': 'mean'
        }).round(2)
        
        return seasonal_stats
    
    def perform_analysis(self):
        """Run all statistical analyses"""
        self.logger.info("Starting statistical analysis...")
        
        # 1. Hypothesis Tests
        hypothesis_results = self.run_hypothesis_tests()
        
        # 2. Correlation Analysis
        corr_matrix, strong_corrs = self.analyze_correlations()
        
        # 3. Seasonality Analysis
        seasonal_patterns = self.analyze_seasonality()
        
        self.logger.info("Statistical analysis completed successfully")
        return hypothesis_results, corr_matrix, strong_corrs, seasonal_patterns

# Perform statistical analysis
analyzer = StatisticalAnalyzer(data_featured)
hyp_results, corr_matrix, strong_corrs, seasonal_patterns = analyzer.perform_analysis()

# Visualize results
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Numerical Features')
plt.show()

print("\nHypothesis Test Results:")
print("------------------------")
for test, results in hyp_results.items():
    print(f"\n{test}:")
    for metric, value in results.items():
        print(f"{metric}: {value:.4f}")

print("\nStrong Correlations:")
print("-------------------")
for corr in strong_corrs[:5]:
    print(f"{corr['var1']} vs {corr['var2']}: {corr['correlation']:.2f}")

print("\nSeasonal Patterns:")
print("----------------")
print(seasonal_patterns)

## 6. Advanced Analytics

### Objectives:
1. Customer Segmentation Analysis
2. Pattern Recognition in Booking Behaviors
3. Anomaly Detection
4. Revenue Impact Analysis
5. Risk Profiling

### Approach:
- Use clustering for customer segmentation
- Implement pattern mining algorithms
- Develop comprehensive risk profiles

In [ ]:
# 6.1 Advanced Analytics Implementation
class AdvancedAnalytics:
    def __init__(self, df):
        self.df = df
        self.logger = logging.getLogger(__name__)
        self.scaler = StandardScaler()
    
    def perform_customer_segmentation(self, n_clusters=4):
        """
        Perform customer segmentation using K-means clustering
        """
        # Select features for clustering
        cluster_features = [
            'adr', 'total_nights', 'lead_time', 'total_guests',
            'is_repeated_guest', 'previous_cancellations'
        ]
        
        # Prepare data for clustering
        X = self.scaler.fit_transform(self.df[cluster_features])
        
        # Perform K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        self.df['customer_segment'] = kmeans.fit_predict(X)
        
        # Analyze segments
        segment_profiles = self.df.groupby('customer_segment').agg({
            'adr': 'mean',
            'total_nights': 'mean',
            'lead_time': 'mean',
            'total_guests': 'mean',
            'is_canceled': 'mean',
            'is_repeated_guest': 'mean'
        }).round(2)
        
        return segment_profiles
    
    def detect_anomalies(self):
        """
        Detect anomalous bookings using statistical methods
        """
        anomalies = {
            'price_anomalies': self.df[np.abs(stats.zscore(self.df['adr'])) > 3],
            'stay_anomalies': self.df[np.abs(stats.zscore(self.df['total_nights'])) > 3],
            'lead_time_anomalies': self.df[np.abs(stats.zscore(self.df['lead_time'])) > 3]
        }
        
        return anomalies
    
    def analyze_revenue_impact(self):
        """
        Analyze revenue impact of different factors
        """
        revenue_analysis = {
            'total_revenue': (self.df['adr'] * self.df['total_nights']).sum(),
            'lost_revenue': (self.df[self.df['is_canceled'] == 1]['adr'] * 
                           self.df[self.df['is_canceled'] == 1]['total_nights']).sum(),
            'revenue_by_segment': self.df.groupby('market_segment').agg({
                'adr': lambda x: (x * self.df['total_nights']).sum(),
                'is_canceled': 'mean'
            }).round(2)
        }
        
        return revenue_analysis
    
    def create_risk_profiles(self):
        """
        Create booking risk profiles
        """
        # Calculate risk scores
        self.df['risk_score'] = (
            0.3 * (self.df['lead_time'] > 90).astype(int) +
            0.2 * (self.df['adr'] > self.df['adr'].mean()).astype(int) +
            0.2 * self.df['previous_cancellations'] +
            0.3 * (self.df['total_nights'] > 7).astype(int)
        )
        
        risk_profiles = self.df.groupby(pd.qcut(self.df['risk_score'], q=4)).agg({
            'is_canceled': 'mean',
            'adr': 'mean',
            'lead_time': 'mean',
            'total_nights': 'mean'
        }).round(2)
        
        return risk_profiles
    
    def run_advanced_analytics(self):
        """
        Run all advanced analytics
        """
        self.logger.info("Starting advanced analytics...")
        
        # 1. Customer Segmentation
        segment_profiles = self.perform_customer_segmentation()
        
        # 2. Anomaly Detection
        anomalies = self.detect_anomalies()
        
        # 3. Revenue Impact Analysis
        revenue_analysis = self.analyze_revenue_impact()
        
        # 4. Risk Profiling
        risk_profiles = self.create_risk_profiles()
        
        self.logger.info("Advanced analytics completed successfully")
        return segment_profiles, anomalies, revenue_analysis, risk_profiles

# Perform advanced analytics
advanced_analyzer = AdvancedAnalytics(data_featured)
segments, anomalies, revenue, risks = advanced_analyzer.run_advanced_analytics()

# Display results
print("Customer Segment Profiles:")
print("-----------------------")
print(segments)

print("\nRevenue Analysis:")
print("---------------")
print(f"Total Revenue: ${revenue['total_revenue']:,.2f}")
print(f"Lost Revenue from Cancellations: ${revenue['lost_revenue']:,.2f}")

print("\nRisk Profiles:")
print("------------")
print(risks)

# Visualize customer segments
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=data_featured,
    x='adr',
    y='lead_time',
    hue='customer_segment',
    palette='deep'
)
plt.title('Customer Segments by ADR and Lead Time')
plt.show()

## 7. Predictive Modeling

In this section, we'll develop and evaluate machine learning models to predict hotel booking cancellations. Our approach includes:

1. Data Preparation
   - Feature selection
   - Train-test split
   - Feature scaling
   
2. Model Development
   - Random Forest Classifier (baseline)
   - XGBoost
   - LightGBM
   
3. Model Evaluation
   - Cross-validation
   - Hyperparameter tuning
   - Performance metrics (accuracy, precision, recall, F1-score)
   - ROC curves and AUC scores
   
4. Model Interpretation
   - Feature importance analysis
   - SHAP values for model explainability
   - Partial dependence plots

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import shap
import logging

class PredictiveModeling:
    def __init__(self, data, target='is_canceled', test_size=0.2, random_state=42):
        self.logger = logging.getLogger(__name__)
        self.data = data
        self.target = target
        self.test_size = test_size
        self.random_state = random_state
        self.models = {}
        self.results = {}
        
    def prepare_data(self):
        """Prepare data for modeling."""
        try:
            # Select features (excluding target and unnecessary columns)
            exclude_cols = ['is_canceled', 'reservation_status', 'reservation_status_date', 'ID']
            feature_cols = [col for col in self.data.columns if col not in exclude_cols]
            
            # Convert categorical variables to dummy variables
            X = pd.get_dummies(self.data[feature_cols], drop_first=True)
            y = self.data[self.target]
            
            # Split data
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
                X, y, test_size=self.test_size, random_state=self.random_state
            )
            
            # Scale features
            scaler = StandardScaler()
            self.X_train_scaled = scaler.fit_transform(self.X_train)
            self.X_test_scaled = scaler.transform(self.X_test)
            
            self.feature_names = X.columns
            self.logger.info(f"Data prepared successfully. Training set shape: {self.X_train.shape}")
            return True
            
        except Exception as e:
            self.logger.error(f"Error in data preparation: {str(e)}")
            return False

# Initialize the modeling class
modeling = PredictiveModeling(data_clean)
modeling.prepare_data()

In [ ]:
def train_and_evaluate_models(modeling):
    """Train and evaluate multiple models."""
    # Random Forest
    rf_params = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestClassifier(random_state=modeling.random_state)
    rf_search = RandomizedSearchCV(rf, rf_params, n_iter=20, cv=5, random_state=modeling.random_state)
    rf_search.fit(modeling.X_train_scaled, modeling.y_train)
    modeling.models['random_forest'] = rf_search.best_estimator_
    
    # XGBoost
    xgb_params = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'n_estimators': [100, 200, 300],
        'min_child_weight': [1, 3, 5]
    }
    
    xgb_model = xgb.XGBClassifier(random_state=modeling.random_state)
    xgb_search = RandomizedSearchCV(xgb_model, xgb_params, n_iter=20, cv=5, random_state=modeling.random_state)
    xgb_search.fit(modeling.X_train_scaled, modeling.y_train)
    modeling.models['xgboost'] = xgb_search.best_estimator_
    
    # LightGBM
    lgb_params = {
        'num_leaves': [31, 50, 70],
        'learning_rate': [0.01, 0.1, 0.3],
        'n_estimators': [100, 200, 300],
        'min_child_samples': [20, 30, 50]
    }
    
    lgb_model = lgb.LGBMClassifier(random_state=modeling.random_state)
    lgb_search = RandomizedSearchCV(lgb_model, lgb_params, n_iter=20, cv=5, random_state=modeling.random_state)
    lgb_search.fit(modeling.X_train_scaled, modeling.y_train)
    modeling.models['lightgbm'] = lgb_search.best_estimator_
    
    # Evaluate models
    results = {}
    for name, model in modeling.models.items():
        y_pred = model.predict(modeling.X_test_scaled)
        y_pred_proba = model.predict_proba(modeling.X_test_scaled)[:, 1]
        
        results[name] = {
            'accuracy': accuracy_score(modeling.y_test, y_pred),
            'precision': precision_score(modeling.y_test, y_pred),
            'recall': recall_score(modeling.y_test, y_pred),
            'f1': f1_score(modeling.y_test, y_pred),
            'auc_roc': roc_auc_score(modeling.y_test, y_pred_proba),
            'confusion_matrix': confusion_matrix(modeling.y_test, y_pred)
        }
    
    modeling.results = results
    return pd.DataFrame(results).round(3)

# Train and evaluate models
results_df = train_and_evaluate_models(modeling)
print("\nModel Evaluation Results:")
print(results_df)

In [ ]:
def interpret_models(modeling):
    """Generate and visualize model interpretations."""
    plt.figure(figsize=(15, 10))
    
    # Random Forest Feature Importance
    rf_importance = pd.DataFrame({
        'feature': modeling.feature_names,
        'importance': modeling.models['random_forest'].feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    plt.subplot(2, 1, 1)
    sns.barplot(data=rf_importance, x='importance', y='feature')
    plt.title('Random Forest Feature Importance')
    plt.xlabel('Importance Score')
    
    # SHAP Values for XGBoost
    explainer = shap.TreeExplainer(modeling.models['xgboost'])
    shap_values = explainer.shap_values(modeling.X_test_scaled)
    
    plt.subplot(2, 1, 2)
    shap.summary_plot(shap_values, modeling.X_test, plot_type='bar', show=False)
    plt.title('SHAP Feature Importance (XGBoost)')
    
    plt.tight_layout()
    plt.show()

    # Generate partial dependence plots for top features
    top_features = rf_importance['feature'].head(3).tolist()
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for i, feature in enumerate(top_features):
        PartialDependenceDisplay.from_estimator(
            modeling.models['random_forest'],
            modeling.X_train,
            [feature],
            ax=axes[i]
        )
        axes[i].set_title(f'Partial Dependence Plot: {feature}')
    
    plt.tight_layout()
    plt.show()

# Interpret models
interpret_models(modeling)

# Print best model insights
best_model = max(modeling.results.items(), key=lambda x: x[1]['f1'])[0]
print(f"\nBest Performing Model: {best_model}")
print("\nBest Model Parameters:")
print(modeling.models[best_model].get_params())

# Save best model predictions for further analysis
best_predictions = modeling.models[best_model].predict(modeling.X_test_scaled)
best_probabilities = modeling.models[best_model].predict_proba(modeling.X_test_scaled)[:, 1]

# Calculate and display confusion matrix
cm = confusion_matrix(modeling.y_test, best_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {best_model}')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### 7.1 Modeling Summary

The predictive modeling section has implemented and evaluated three different models:
1. Random Forest Classifier
2. XGBoost
3. LightGBM

Key findings from the modeling process:
- Model Performance: Comparison of accuracy, precision, recall, and F1-scores
- Feature Importance: Identification of key factors influencing booking cancellations
- Model Interpretability: SHAP values and partial dependence plots revealing the relationship between features and cancellation probability

Next, we'll translate these technical insights into actionable business recommendations in the Business Insights section.

## 8. Business Insights

In this section, we'll synthesize all our analyses to provide actionable business recommendations:
1. Key Findings from Data Analysis
2. Revenue Impact Assessment
3. Risk Mitigation Strategies
4. Actionable Recommendations
5. Implementation Plan

In [ ]:
class BusinessInsights:
    def __init__(self, data, modeling):
        self.data = data
        self.modeling = modeling
        self.insights = {}
    
    def analyze_key_findings(self):
        """Analyze and summarize key findings from all analyses."""
        # Cancellation patterns
        cancellation_rate = self.data['is_canceled'].mean() * 100
        
        # Seasonal patterns
        monthly_cancellations = self.data.groupby('arrival_date_month')['is_canceled'].mean() * 100
        peak_cancellation_month = monthly_cancellations.idxmax()
        
        # Lead time analysis
        avg_lead_time = self.data.groupby('is_canceled')['lead_time'].mean()
        
        # Customer segments
        segment_cancellations = self.data.groupby('market_segment')['is_canceled'].mean() * 100
        high_risk_segment = segment_cancellations.idxmax()
        
        self.insights['key_findings'] = {
            'cancellation_rate': cancellation_rate,
            'peak_cancellation_month': peak_cancellation_month,
            'lead_time_difference': avg_lead_time[1] - avg_lead_time[0],
            'high_risk_segment': high_risk_segment
        }
        
    def calculate_revenue_impact(self):
        """Calculate revenue impact of cancellations."""
        # Calculate potential revenue loss
        adr_cancelled = self.data[self.data['is_canceled'] == 1]['adr'].sum()
        total_cancelled_bookings = self.data['is_canceled'].sum()
        
        # Calculate revenue recovery potential
        best_model = max(self.modeling.results.items(), key=lambda x: x[1]['f1'])[0]
        model_accuracy = self.modeling.results[best_model]['accuracy']
        potential_savings = adr_cancelled * model_accuracy
        
        self.insights['revenue_impact'] = {
            'total_revenue_loss': adr_cancelled,
            'cancelled_bookings': total_cancelled_bookings,
            'potential_savings': potential_savings
        }
    
    def generate_recommendations(self):
        """Generate actionable recommendations based on analysis."""
        feature_importance = pd.DataFrame({
            'feature': self.modeling.feature_names,
            'importance': self.modeling.models['random_forest'].feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Generate recommendations based on top features
        recommendations = []
        for _, row in feature_importance.head().iterrows():
            feature = row['feature']
            if 'lead_time' in feature.lower():
                recommendations.append("Implement dynamic pricing based on lead time")
            elif 'market_segment' in feature.lower():
                recommendations.append("Develop targeted retention strategies for high-risk segments")
            elif 'deposit' in feature.lower():
                recommendations.append("Optimize deposit policies based on booking characteristics")
            elif 'month' in feature.lower():
                recommendations.append("Adjust pricing and policies for seasonal patterns")
                
        self.insights['recommendations'] = recommendations
        
    def create_implementation_plan(self):
        """Create an implementation plan for recommendations."""
        self.insights['implementation_plan'] = {
            'short_term': [
                "Update deposit policies",
                "Implement email confirmation system",
                "Train staff on new procedures"
            ],
            'medium_term': [
                "Develop dynamic pricing system",
                "Create customer segment strategies",
                "Implement automated reminder system"
            ],
            'long_term': [
                "Build predictive cancellation system",
                "Develop loyalty program",
                "Implement revenue optimization system"
            ]
        }

# Generate business insights
insights = BusinessInsights(data_clean, modeling)
insights.analyze_key_findings()
insights.calculate_revenue_impact()
insights.generate_recommendations()
insights.create_implementation_plan()

# Display key findings
print("\nKey Findings:")
for metric, value in insights.insights['key_findings'].items():
    print(f"{metric.replace('_', ' ').title()}: {value:.2f}" if isinstance(value, float) else f"{metric.replace('_', ' ').title()}: {value}")

print("\nRevenue Impact:")
for metric, value in insights.insights['revenue_impact'].items():
    print(f"{metric.replace('_', ' ').title()}: ${value:,.2f}")

print("\nTop Recommendations:")
for i, rec in enumerate(insights.insights['recommendations'], 1):
    print(f"{i}. {rec}")

print("\nImplementation Plan:")
for phase, actions in insights.insights['implementation_plan'].items():
    print(f"\n{phase.replace('_', ' ').title()} Actions:")
    for action in actions:
        print(f"- {action}")

# Visualize key insights
plt.figure(figsize=(15, 10))

# Plot 1: Revenue Impact
plt.subplot(2, 2, 1)
revenue_data = [insights.insights['revenue_impact']['total_revenue_loss'],
                insights.insights['revenue_impact']['potential_savings']]
plt.bar(['Total Revenue Loss', 'Potential Savings'], revenue_data)
plt.title('Revenue Impact Analysis')
plt.ylabel('Amount ($)')

# Plot 2: Implementation Timeline
plt.subplot(2, 2, 2)
timeline_data = [len(phase) for phase in insights.insights['implementation_plan'].values()]
plt.bar(['Short Term', 'Medium Term', 'Long Term'], timeline_data)
plt.title('Implementation Plan Timeline')
plt.ylabel('Number of Actions')

plt.tight_layout()
plt.show()

## 9. Final Review and Production Readiness

### Code Quality and Documentation
- Modular, object-oriented design with clear class responsibilities
- Comprehensive error handling and logging
- Well-documented functions and classes
- Consistent coding style and naming conventions

### Reproducibility
- Environment setup and dependency management
- Data versioning and preprocessing pipeline
- Modular structure for easy maintenance
- Clear execution flow

### Production Readiness
- Scalable data processing
- Robust error handling
- Performance optimization
- Monitoring capabilities

### Future Improvements
1. Model Deployment
   - API development for model serving
   - Monitoring system for model performance
   - Regular model retraining pipeline

2. Additional Features
   - Real-time prediction capabilities
   - Integration with booking systems
   - Automated reporting system

3. Optimization
   - Feature selection optimization
   - Model performance tuning
   - Processing pipeline optimization

## 10. Deployment Steps (Optional)

### Model Deployment Pipeline
1. Model Serialization
   - Save trained models using joblib/pickle
   - Version control for model artifacts
   - Documentation of model parameters and performance

2. API Development
   - FastAPI/Flask REST API
   - Input validation
   - Error handling
   - Authentication and rate limiting

3. Infrastructure Setup
   - Container orchestration (Docker/Kubernetes)
   - Load balancing
   - Auto-scaling configuration
   - Monitoring and logging setup

4. CI/CD Pipeline
   - Automated testing
   - Model validation
   - Deployment automation
   - Rollback procedures

5. Monitoring System
   - Model performance metrics
   - Data drift detection
   - Resource utilization
   - Alert system

6. Documentation
   - API documentation
   - Deployment procedures
   - Maintenance guides
   - Troubleshooting procedures